In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch


In [ ]:
def evolution(x, t, l = 1):
   return  np.exp(l * t) * x 

def evolution_advanced(x, t, l = 1):
    if x > 0 and x < 0.5:
        return np.exp(l * t) * x 
    else:
        return np.exp(2 * l * t) * x
    


def logistic(x_0,r = 3.2, reps = 600):
    x = np.zeros(reps)
    x[0] = x_0
    for i in range(reps - 1):
        x[i + 1] = r * x[i] * (1 - x[i])
    return x

In [ ]:
def traj(x, l = 1):
    traj = np.zeros((100), dtype = np.float64)
    for i in np.arange(100):
        traj[i] = evolution(x,i,l)
    return traj

In [ ]:
def le(x,y,t):
    d = np.abs(x - y)
    
    d = d/d[0]
    d = np.log(d)
    

    d = d/t
    return d

t = np.linspace(0,10,3)
x = evolution(0.,t)
y = evolution(0.1,t)

print(x)
print(y)

# r = 2 #logistic parameter
# x = logistic(0.1,r)
# y = logistic(0.1001,r)
# t = np.arange(600)

le_val = le(x,y,t)
plt.plot(t,le_val)
plt.show()

In [ ]:
r = 3.8
x = logistic(0.5,r)
y = logistic(0.5001,r)
t = np.arange(600)

le_val = le(x,y,t)

plt.plot(t,le_val)
plt.show()

In [ ]:
x_amount = 100
reps = 60
eps = 0.1
t = np.arange(reps)

# ini_values = np.linspace(0.4,0.5,x_amount)
ini_values = np.random.rand(x_amount)

traj = np.zeros((reps,x_amount))
traj[0,:] = ini_values

for i in range(x_amount):
    traj[:,i] = evolution_advanced(traj[0,i],t)

print(traj)


le_val = np.zeros_like(traj)

for i in range(x_amount):
    count = 0
    for j in range(x_amount):
        if (np.abs(traj[0,i] - traj[0,j]) < eps and not(i == j)):
            count += 1
            le_val[:,i] += le(traj[:,i],traj[:,j],t)
            print('avg le update with count ',count,' and value ',le_val[-1,i])
    if count > 1:
        le_val[:,i] = le_val[:,i]/count
            

# print(traj)
print(le_val[-1,:])

In [ ]:
indices = np.argsort(ini_values)

ini_values = ini_values[indices]
le_final = le_val[-1,:]
le_final = le_final[indices]

plt.scatter(ini_values, le_final)
plt.show()

Now i want to check how to make a two dimensional plot

In [ ]:


def test_2d(x, t, l = 1):
    return (torch.linalg.norm(x, dim = -1) < 1).float()


x_amount = 100

x = torch.linspace(-1,1,x_amount)
y = torch.linspace(-1,1,x_amount)
X, Y = torch.meshgrid(x, y)

inputs = torch.stack([X,Y], dim=-1)
print(inputs.shape)



# Compute the norm for each pair of corresponding elements from X and Y
Z = test_2d(inputs,1)
print(Z)

# Create heatmap using imshow
plt.imshow(Z, origin='lower', extent=(-1, 1, -1, 1), cmap='viridis')
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
a = torch.tensor([1,2])
b = torch.linspace(0,1,5)

a = a[:,None]
print(a*b)

In [ ]:
def evolution_2d(x, t, l=1.):
    norm_x = torch.linalg.norm(x, dim = -1, keepdim=True)  # shape (x_amount,x_amount,1)
    indices_below = (norm_x < 1).expand_as(x)  # shape (x_amount,x_amount,dim_x)

    t = t[None, None, None, :]  # shape (1,1,1,3)
    x_extended = x.unsqueeze(-1).expand(-1, -1, -1, len(t))  # shape (x_amount,x_amount,dim_x,len_t)

    result_below = torch.exp(-l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)
    result_above = torch.exp(l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)

    # Use the boolean mask to select values from result_below or result_above
    result = torch.where(indices_below.unsqueeze(-1), result_below, result_above)  # shape (x_amount,x_amount,dim_x,len_t)

    return result


    

x_amount = 10

x = torch.linspace(-1,1,x_amount)
y = torch.linspace(-1,1,x_amount)
X, Y = torch.meshgrid(x, y)

inputs = torch.stack([X,Y], dim=-1)
print(f'{inputs = }')
print(inputs.shape)



# Compute the norm for each pair of corresponding elements from X and Y
t = torch.linspace(0,1,3)
Z = evolution_2d(inputs,t)
print(Z.shape)

Z = Z[:,:,:,-1] #reduce to last time instance T
Z = torch.linalg.norm(Z, dim = -1) #compute norm of each x(T)

# Create heatmap using imshow
plt.imshow(Z, origin='lower', extent=(-1, 1, -1, 1), cmap='viridis')
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
def evolution_orientation(x, t, l=1.):
    indices_below = (x[:,:,0:1] > 0) & (x[:,:,1:2] < 0)
    indices_below = indices_below.expand_as(x) # shape (x_amount,x_amount,dim_x)

    t = t[None, None, None, :]  # shape (1,1,1,3)
    x_extended = x.unsqueeze(-1).expand(-1, -1, -1, len(t))  # shape (x_amount,x_amount,dim_x,len_t)

    result_below = torch.exp(-l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)
    result_above = torch.exp(l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)

    # Use the boolean mask to select values from result_below or result_above
    result = torch.where(indices_below.unsqueeze(-1), result_below, result_above)  # shape (x_amount,x_amount,dim_x,len_t)

    return result

x_amount = 10

x = torch.linspace(-1,1,x_amount)
y = torch.linspace(-1,1,x_amount)
X, Y = torch.meshgrid(x, y)

inputs = torch.stack([X,Y], dim=-1)
print(f'{inputs = }')
print(inputs.shape)



# Compute the norm for each pair of corresponding elements from X and Y
t = torch.linspace(0,1,3)
Z = evolution_orientation(inputs,t)
print(Z.shape)

Z = Z[:,:,:,-1] #reduce to last time instance T
Z = torch.linalg.norm(Z, dim = -1) #compute norm of each x(T)

# Create heatmap using imshow
plt.imshow(Z, origin='lower', extent=(-1, 1, -1, 1), cmap='viridis')
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
'''
input: two trajectories x(t), y(t) where each has size (x_dim,len_t)
output: maximum lyapunov exponent for each time t, size (1,len_t)'''
def le(x,y,t):
    d = x - y
    d = torch.linalg.norm(x - y, dim = 0)
    print(f'{d.shape = }')
    d = d/d[0]
    d = np.log(d)
    

    d = d/t
    return d



'''
Compute the maximum Lyapunov exponent with initial value tolerance eps
input: 
trajectories of size (x_amount,x_amount,x_dim,time_dim)
each trajectory has initial value x_0 = traj[i,j,:,0]

output:
MLE for each trajectory as averaged LE for all initial values that is eps close to the initial value
output size (x_amount,x_amount,t_dim)
'''
def MLE(traj, t, eps = 0.1):
    x_amount = traj.size(0)
    x_amount, y_amount, x_dim, t_dim = traj.shape
    le_val = torch.zeros((x_amount, y_amount, t_dim)) 
    print(f'{le_val.shape = }')
    for i in range(x_amount):
        for j in range(y_amount):
            count = 0
            for i_comp in range(x_amount):
                for j_comp in range(y_amount):
                    if (torch.norm(traj[i,j,:,0] - traj[i_comp,j_comp,:,0]) < eps and not(i == i_comp and j == j_comp)):
                        count += 1
                        le_val[i,j] += le(traj[i,j,:,:],traj[i_comp,j_comp,:,:],t)
                        print('avg le update with count ',count,' and value ',le_val[i,j,-1])
            if count > 1:
                le_val[i,j,:] = le_val[i,j,:]/count
    return le_val

# x1 = inputs[0,0,:]
# x2 = inputs[1,0,:]

# print(x1,x2)

# le(x1,x2,1)
t = torch.linspace(0,1,10)
evo = evolution_2d(inputs,t)
print(f'{evo.shape = }')

evo1 = evo[0,0,:]
evo2 = evo[1,1,:]

print(f'{evo1.shape = }')

le(evo1,evo2, t)

In [ ]:
x_amount = 30
eps = 0.1
x = torch.linspace(-1,1,x_amount)
y = torch.linspace(-1,1,x_amount)
X, Y = torch.meshgrid(x, y)

inputs = torch.stack([X,Y], dim=-1)

# Compute the norm for each pair of corresponding elements from X and Y
t = torch.linspace(0,1,2)
Z = evolution_2d(inputs,t)
print(Z.shape)

output = MLE(Z,t,eps)
print(output)

output = output[:,:,-1] #reduce to last time instance T
# Create heatmap using imshow
plt.imshow(output, origin='lower', extent=(-1, 1, -1, 1), cmap='viridis')
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
x = torch.zeros((1,2,3))
x_amount = 3
for i in range(x_amount):
    for j in range(x_amount):
        print(i,j)


In [ ]:
def le_tensor(x, y, t):
    d = torch.linalg.norm(x - y, dim=-2)
    print(d.size())
    d = d / torch.linalg.norm(d[..., 0], dim=-1).unsqueeze(-1)
    d = torch.log(d)
    return (d / t).unsqueeze(-1)


def MLE_tensor(traj, t, eps = 0.1):
    x_amount, y_amount, x_dim, t_dim = traj.shape
    
    x = traj.unsqueeze(2).unsqueeze(3)  # shape (x_amount, y_amount, 1, 1, x_dim, t_dim)
    y = traj.unsqueeze(0).unsqueeze(1)  # shape (1, 1, x_amount, y_amount, x_dim, t_dim)
    
    
    # Compute pairwise distances for initial conditions
    dists = torch.linalg.norm(x[..., :, 0] - y[..., :, 0], dim=4)
    
    # Create mask for pairs with close initial conditions (exclude self-comparisons)
    mask = (dists < eps) & (torch.arange(x_amount).view(-1, 1) != torch.arange(y_amount))
    
    # Compute the Lyapunov exponent for each pair of trajectories
    lyap_exps = le_tensor(x, y, t)  # shape (x_amount, y_amount, x_amount, y_amount, t_dim, 1)
    print(f'{lyap_exps.shape=}')
    
    # Compute the mean Lyapunov exponent for each trajectory, excluding pairs that are too far apart
    mask_expanded = mask.unsqueeze(-1).unsqueeze(-1)  # shape (x_amount, y_amount, x_amount, y_amount, 1, 1)
    le_val = (lyap_exps * mask_expanded).sum(dim=(2, 3)) / mask_expanded.sum(dim=(2, 3))
    
    return le_val.squeeze(-1)  # shape (x_amount, y_amount, t_dim)






x_amount = 5
eps = 0.1
x = torch.linspace(-1,1,x_amount)
y = torch.linspace(-1,1,x_amount)
X, Y = torch.meshgrid(x, y)

inputs = torch.stack([X,Y], dim=-1)

# Compute the norm for each pair of corresponding elements from X and Y
t = torch.linspace(0,1,3)
Z = evolution_2d(inputs,t)
print(Z.shape)

output = MLE_tensor(Z,t,eps)

output = output[:,:,-1] #reduce to last time instance T
print('output size = ', output.size())
print(output)
# Create heatmap using imshow
plt.imshow(output, origin='lower', extent=(-1, 1, -1, 1), cmap='viridis')
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
test = torch.zeros(size = (1,2,4))
test[:,0,:] = 1
test[:,1,:] = 2
print('test = ', test)

print(test.sum(dim = 2))

print(test / torch.linalg.norm(test[...,0], dim = -1))